In [ ]:
import pandas as pd
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import tiktoken
import time
import random
import nltk
from tqdm import tqdm
import torch
import os
import json
import random
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import functions as fn

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
url = "https://raw.githubusercontent.com/chen700564/RGB/master/data/en.json"
data = fn.process_json(url)
# data = random.sample(data, 5)
queries = [item["query"] for item in data]
answers = [item["answer"][0] for item in data]

In [ ]:
# Initialize time
start_time = time.time()

noise_thresholds = {
    'Noise_0': 0.0,
    'Noise_20': 0.20,
    'Noise_40': 0.40,
    'Noise_60': 0.60,
    'Noise_80': 0.80,
    'Noise_100': 1.0
}

# Set GPU
device = 0 if torch.cuda.is_available() else -1

# Set up models
ext_model_1 = "timpal0l/mdeberta-v3-base-squad2"
ext_model_2 = "deepset/xlm-roberta-large-squad2"
ext_model_3 = "google-bert/bert-base-multilingual-cased"
ext_model_4 = "mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp"
ext_model_5 = "deepset/roberta-base-squad2"
ext_model_6 = "distilbert/distilbert-base-cased-distilled-squad"
ext_model_7 = "deepset/electra-base-squad2"
ext_model_8 = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"

# models = [ ext_model_2, ext_model_4, ext_model_6, ext_model_8 ]
models = [ ext_model_6, ext_model_8 ]
separator = " <|> "

# Number of experiments
num_experiments = 5

for model in models:
    tokenizer = AutoTokenizer.from_pretrained(model)
    loaded_model = AutoModelForQuestionAnswering.from_pretrained(model)
    qa_pipeline = pipeline("question-answering", model=loaded_model, tokenizer=tokenizer, device=device)

    for exp_num in range(1, num_experiments + 1):
        # Set a unique seed for each experiment to ensure replicability
        random.seed(2024 + exp_num)

        results = []
        for query, positive_context, negative_context, answer in tqdm(zip(queries, [item["positive"] for item in data], [item["negative"] for item in data], answers), total=len(queries)):
            result = {
                'Query': query,
                'Correct Answer': answer,
            }
            for noise_level, label in noise_thresholds.items():
                mixed_context = fn.create_mixed_context(positive_context, negative_context, label, 1600, separator)
                context_result = fn.process_context(mixed_context, qa_pipeline, query, separator)
                context_result.pop('Appended Context', None)
                context_result.pop('Context Interval', None)
                result.update({noise_level + ' ' + k: v for k, v in context_result.items()})
                result[f'Jaccard {noise_level}'] = fn.apply_jaccard(result, f'{noise_level} Predicted Answer', 'Correct Answer')
                result[f'Cosine {noise_level}'] = fn.apply_cosine(result, f'{noise_level} Predicted Answer', 'Correct Answer')
                result[f'EM {noise_level}'] = fn.apply_exact_match(result, f'{noise_level} Predicted Answer', 'Correct Answer')
                result[f'EM - 2V {noise_level}'] = fn.apply_exact_match_2v(result, f'{noise_level} Predicted Answer', 'Correct Answer')

            results.append(result)

        results_df = pd.DataFrame(results)

        # Save results to a file with the experiment number and model name
        filename = f"/results/ext/exp_{exp_num}_{model.split('/')[1]}.json"
        results_df.to_json(filename, orient='records', lines=True)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 300/300 [16:37<00:00,  3.33s/it]


In [ ]:
# input_paths = '/content/drive/My Drive/tornado-tasks/results/ext/'
# files = os.listdir(input_paths)

# for file in files:
#     if file.endswith('.json'):
#         model_name = file.split('.')[0]
#         full_path = os.path.join(input_paths, file)
#         for data_chunk in fn.read_json_in_chunks(full_path, fn.cols_to_use, chunk_size=1000):
#             with open(f'/content/drive/My Drive/tornado-tasks/text/ext/{model_name}.txt', 'a') as output_file:
#                 print(model_name)
#                 for part in fn.format_results(data_chunk):
#                     output_file.write(part + '\n')

In [ ]:
threshold = 0.8
input_paths = '/results/ext/'
output_file = '/content/drive/My Drive/tornado-tasks/metrics/ext/all_metrics.xlsx'

all_results = {}

files = os.listdir(input_paths)

for i, file in enumerate(tqdm(files, desc="Processing files"), start=1):
    if file.endswith('.json'):
        experiment_num = int(file.split('_')[1])  # Extraer el número de experimento del nombre del archivo
        model_name = file.split('_')[2].replace('.json', '')  # Extraer el nombre del modelo
        sheet_name = model_name
        print(f"Processing file {i}/{len(files)}: {file}")
        input_path = os.path.join(input_paths, file)
        result_df = fn.compute_metrics(input_path, threshold)  # No pasar experiment_num
        result_df.insert(0, 'Experiment Number', experiment_num)  # Insertar columna del número de experimento

        if sheet_name not in all_results:
            all_results[sheet_name] = result_df
        else:
            all_results[sheet_name] = pd.concat([all_results[sheet_name], result_df], ignore_index=True)

with pd.ExcelWriter(output_file) as writer:
    for sheet_name, result_df in all_results.items():
        result_df.to_excel(writer, sheet_name=sheet_name, index=False)

Processing files:   0%|          | 0/20 [00:00<?, ?it/s]

Processing file 1/20: exp_1_xlm-roberta-large-squad2.json


Processing files:   5%|▌         | 1/20 [05:52<1:51:30, 352.13s/it]

Processing file 2/20: exp_2_xlm-roberta-large-squad2.json


Processing files:  10%|█         | 2/20 [11:40<1:45:02, 350.12s/it]

...

Processing files:  90%|█████████ | 18/20 [1:43:02<11:21, 340.73s/it]

Processing file 19/20: exp_4_bert-large-uncased-whole-word-masking-finetuned-squad.json


Processing files:  95%|█████████▌| 19/20 [1:48:43<05:40, 340.81s/it]

Processing file 20/20: exp_5_bert-large-uncased-whole-word-masking-finetuned-squad.json


Processing files: 100%|██████████| 20/20 [1:54:22<00:00, 343.15s/it]
/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [ ]:
input_file = '/metrics/ext/all_metrics.xlsx'
output_file = '/metrics/ext/final_metrics.xlsx'

# Leer el archivo Excel original
excel_data = pd.read_excel(input_file, sheet_name=None)

final_results = {}

# Procesar cada hoja del archivo Excel
for sheet_name, df in excel_data.items():
    # Asegurarse de que la columna 'Experiment Number' esté presente
    if 'Experiment Number' not in df.columns:
        raise ValueError(f"'Experiment Number' column not found in sheet {sheet_name}")

    # Identificar las métricas y niveles de ruido
    metrics = df['Metric'].unique()
    noise_levels = ['Noise_0', 'Noise_20', 'Noise_40', 'Noise_60', 'Noise_80', 'Noise_100']

    # Crear DataFrame para resultados finales
    result_data = {
        'Metric': metrics,
    }

    # Calcular la media y desviación estándar para cada métrica y nivel de ruido
    for noise_level in noise_levels:
        result_data[f'{noise_level}_Mean'] = []
        result_data[f'{noise_level}_Std'] = []
        for metric in metrics:
            metric_df = df[df['Metric'] == metric]
            result_data[f'{noise_level}_Mean'].append(metric_df[noise_level].mean())
            result_data[f'{noise_level}_Std'].append(metric_df[noise_level].std())

    result_df = pd.DataFrame(result_data)
    final_results[sheet_name] = result_df

# Guardar los resultados en un nuevo archivo Excel
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, result_df in final_results.items():
        result_df.to_excel(writer, sheet_name=sheet_name, index=False)

/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
